Machine Learning Assignment 2
Active Learning

Group Number : 29

Group Members:
Aquib Junaid Razack (2017A2PS0704P)
Shyam Sundar P I (2016A4PS0378P)
Praveen Ravirathinam (2017A7PS1174P)

Wine Classification
Question 1 and 2

# Setting up the environment

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

In [5]:
from sklearn.cluster import KMeans

# Preprocessing the data

In [6]:
wine = load_wine()

In [7]:
train = pd.DataFrame(wine.data, columns=wine.feature_names)

In [8]:
target = pd.DataFrame(wine.target, columns = ['Class'])

In [9]:
scal = MinMaxScaler()

In [10]:
train = scal.fit_transform(train)

In [11]:
train = pd.DataFrame(train, columns = wine.feature_names)

In [12]:
#Splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size = 0.20, random_state = 69)

In [13]:
#Splitting train into labelled and unlabelled
X_train_init, X_unlabelled, y_train_init, oracle = train_test_split(X_train, y_train, test_size = 0.95, random_state = 69)

# Uncertainty Sampling

In [14]:
def uncertainty(model,unlabelled,oracle,mode,AL_type):
    if AL_type.lower() == "stream":
        new_X_train = pd.DataFrame(columns = unlabelled.columns)
        new_y_train = pd.DataFrame(columns = oracle.columns)
        if mode == 1:
            for i, row in unlabelled.iterrows():
                prob = model.predict_proba(pd.DataFrame(row).T).max()
                if prob < 0.38:
                    new_X_train = new_X_train.append(row.T)
                    new_y_train = new_y_train.append(pd.DataFrame(oracle.loc[i,:]).T)
            return new_X_train, new_y_train
        if mode == 2:
            for i, row in unlabelled.iterrows():
                prob = np.sort(model.predict_proba(pd.DataFrame(row).T))[0][2] - np.sort(model.predict_proba(pd.DataFrame(row).T))[0][1]
                if prob < 0.03:
                    new_X_train = new_X_train.append(row.T)
                    new_y_train = new_y_train.append(pd.DataFrame(oracle.loc[i,:]).T)
            return new_X_train, new_y_train
        if mode == 3:
            for i, row in unlabelled.iterrows():
                prob = model.predict_proba(pd.DataFrame(row).T)
                ent = -np.sum(prob*np.log(prob))
                if ent > 1.09:
                    new_X_train = new_X_train.append(row.T)
                    new_y_train = new_y_train.append(pd.DataFrame(oracle.loc[i,:]).T)
            return new_X_train, new_y_train
        
    if AL_type.lower() == "pool":
        new_X_train = pd.DataFrame(columns = unlabelled.columns)
        new_y_train = pd.DataFrame(columns = oracle.columns)
        if mode == 1:
            prob = pd.DataFrame(model.predict_proba(unlabelled))
            prob_max_idx = np.argsort(prob.max(axis = 1))
            idx = prob_max_idx[:20]
            new_X_train = unlabelled.iloc[idx,:]
            new_y_train = oracle.iloc[idx,:]
            return new_X_train, new_y_train
        if mode == 2:
            prob = model.predict_proba(unlabelled)
            idx = np.argsort(np.sort(prob)[:,2] - np.sort(prob)[:,1])[:20]
            new_X_train = unlabelled.iloc[idx,:]
            new_y_train = oracle.iloc[idx,:]
            return new_X_train, new_y_train
        if mode == 3:
            prob = model.predict_proba(unlabelled)
            idx = np.argsort(-np.sum(prob*np.log(prob),axis = 1))[-20:]
            new_X_train = unlabelled.iloc[idx,:]
            new_y_train = oracle.iloc[idx,:]
            return new_X_train, new_y_train

# Query by Committee 

In [15]:
def qbc(committee, unlabelled, oracle, mode, AL_type):
    if AL_type.lower() == "stream":
        new_X_train = pd.DataFrame(columns = unlabelled.columns)
        new_y_train = pd.DataFrame(columns = oracle.columns)  
        if mode == 1:
            pred = []
            for i, row in unlabelled.iterrows():
                for j in range(7):
                    pred.append(committee[j].predict(pd.DataFrame(row).T)[0])

                x = pred.count(0)/len(pred) * np.log(pred.count(0)/len(pred))
                y = pred.count(1)/len(pred) * np.log(pred.count(1)/len(pred))
                z = pred.count(2)/len(pred) * np.log(pred.count(2)/len(pred))
                if pred.count(0) == 0:
                    x = 0
                if pred.count(1) == 0:
                    y = 0
                if pred.count(2) == 0:
                    z = 0
                ent = -(x + y + z)
                if ent > 1:
                    new_X_train = new_X_train.append(row.T)
                    new_y_train = new_y_train.append(pd.DataFrame(oracle.loc[i,:]).T)
            return new_X_train, new_y_train
        if mode == 2: 
            pred = []
            for i, row in unlabelled.iterrows():
                for j in range(7):
                    pred.append(committee[j].predict(pd.DataFrame(row).T)[0])

                x = pred.count(0) * np.log(pred.count(0)/len(pred))
                y = pred.count(1) * np.log(pred.count(1)/len(pred))
                z = pred.count(2) * np.log(pred.count(2)/len(pred))
                if pred.count(0) == 0:
                    x = 0
                if pred.count(1) == 0:
                    y = 0
                if pred.count(2) == 0:
                    z = 0
                kld = -(x + y + z)
                if kld > 700:
                    new_X_train = new_X_train.append(row.T)
                    new_y_train = new_y_train.append(pd.DataFrame(oracle.loc[i,:]).T)
            return new_X_train, new_y_train
    
    if AL_type.lower() == "pool":
        new_X_train = pd.DataFrame(columns = unlabelled.columns)
        new_y_train = pd.DataFrame(columns = oracle.columns)  
        if mode == 1:
            pred = []
            ent = []
            for j in range(7):
                pred.append(committee[j].predict(unlabelled))
            pred = np.array(pred)

            for i in range(len(X_unlabelled)):
                c_pred = pd.DataFrame(pred).iloc[:,i].tolist()
                x = c_pred.count(0)/len(c_pred) * np.log(c_pred.count(0)/len(c_pred))
                y = c_pred.count(1)/len(c_pred) * np.log(c_pred.count(1)/len(c_pred))
                z = c_pred.count(2)/len(c_pred) * np.log(c_pred.count(2)/len(c_pred))
                if c_pred.count(0) == 0:
                    x = 0
                if c_pred.count(1) == 0:
                    y = 0
                if c_pred.count(2) == 0:
                    z = 0
                ent.append(-(x + y + z))
            ent = np.array(ent)    
            idx = np.argsort(ent,axis = 0)[-20:]
            new_X_train = unlabelled.iloc[idx,:]
            new_y_train = oracle.iloc[idx,:]
            return new_X_train, new_y_train

        if mode == 2: 
            pred = []
            ent = []
            for j in range(7):
                pred.append(committee[j].predict(unlabelled))
            pred = np.array(pred)

            for i in range(len(X_unlabelled)):
                c_pred = pd.DataFrame(pred).iloc[:,i].tolist()
                x = c_pred.count(0) * np.log(c_pred.count(0)/len(c_pred))
                y = c_pred.count(1) * np.log(c_pred.count(1)/len(c_pred))
                z = c_pred.count(2) * np.log(c_pred.count(2)/len(c_pred))
                if c_pred.count(0) == 0:
                    x = 0
                if c_pred.count(1) == 0:
                    y = 0
                if c_pred.count(2) == 0:
                    z = 0
                ent.append(-(x + y + z))
            ent = np.array(ent)    
            idx = np.argsort(ent,axis = 0)[-20:]
            new_X_train = unlabelled.iloc[idx,:]
            new_y_train = oracle.iloc[idx,:]
            return new_X_train, new_y_train

# Diversity Sampling

In [16]:
def div(model, unlabelled, oracle, AL_type):
    if AL_type.lower() == "pool":
        new_X_train = pd.DataFrame(columns = unlabelled.columns)
        new_y_train = pd.DataFrame(columns = oracle.columns)
        kmeans = KMeans(n_clusters = 3, random_state = 69)
        kmeans.fit(unlabelled)
        unlabelled['Cluster number'] = pd.DataFrame(kmeans.labels_).values

        X_zero = unlabelled[unlabelled['Cluster number'] == 0]
        X_one = unlabelled[unlabelled['Cluster number'] == 1]
        X_two = unlabelled[unlabelled['Cluster number'] == 2]
        oracle_X_zero = oracle.loc[X_zero.index,:]
        oracle_X_one = oracle.loc[X_one.index,:]
        oracle_X_two = oracle.loc[X_two.index,:]

        rand_zero = np.random.randint(0, len(X_zero), 10)
        rand_one = np.random.randint(0, len(X_one), 10)
        rand_two = np.random.randint(0, len(X_two), 10)

        new_X_zero = X_zero.iloc[rand_zero,:]
        new_X_one = X_one.iloc[rand_one,:]
        new_X_two = X_two.iloc[rand_two,:]
        new_y_zero = oracle_X_zero.iloc[rand_zero,:]
        new_y_one = oracle_X_one.iloc[rand_one,:]
        new_y_two = oracle_X_two.iloc[rand_two,:]

        new_X_train = new_X_zero.append(new_X_one).append(new_X_two)
        new_y_train = new_y_zero.append(new_y_one).append(new_y_two)
        new_X_train = new_X_train.drop(columns = ['Cluster number'])

        new_X_train, new_y_train = uncertainty(model, new_X_train, new_y_train, 1, AL_type)

        return new_X_train, new_y_train
    
    if AL_type.lower() == "stream":
        new_X_train = pd.DataFrame(columns = unlabelled.columns)
        new_y_train = pd.DataFrame(columns = oracle.columns)
        kmeans = KMeans(n_clusters = 3, random_state = 69)
        kmeans.fit(unlabelled)
        unlabelled['Cluster number'] = pd.DataFrame(kmeans.labels_).values

        X_zero = unlabelled[unlabelled['Cluster number'] == 0]
        X_one = unlabelled[unlabelled['Cluster number'] == 1]
        X_two = unlabelled[unlabelled['Cluster number'] == 2]
        oracle_X_zero = oracle.loc[X_zero.index,:]
        oracle_X_one = oracle.loc[X_one.index,:]
        oracle_X_two = oracle.loc[X_two.index,:]

        rand_zero = np.random.randint(0, len(X_zero), 10)
        rand_one = np.random.randint(0, len(X_one), 10)
        rand_two = np.random.randint(0, len(X_two), 10)

        new_X_zero = X_zero.iloc[rand_zero,:]
        new_X_one = X_one.iloc[rand_one,:]
        new_X_two = X_two.iloc[rand_two,:]
        new_y_zero = oracle_X_zero.iloc[rand_zero,:]
        new_y_one = oracle_X_one.iloc[rand_one,:]
        new_y_two = oracle_X_two.iloc[rand_two,:]

        new_X_train = new_X_zero.append(new_X_one).append(new_X_two)
        new_y_train = new_y_zero.append(new_y_one).append(new_y_two)
        new_X_train = new_X_train.drop(columns = ['Cluster number'])

        new_X_train, new_y_train = uncertainty(model, new_X_train, new_y_train, 1, AL_type)

        return new_X_train, new_y_train

# Main Function

In [17]:
model = LogisticRegression()

model.fit(X_train_init,y_train_init)

prediction = model.predict(X_test)
accuracy_score(y_test, prediction)

0.6388888888888888

In [18]:
committee = []
committee.append(LogisticRegression())
committee.append(DecisionTreeClassifier())
committee.append(RandomForestClassifier())
committee.append(SVC())
committee.append(GaussianNB())
committee.append(LinearDiscriminantAnalysis())
committee.append(KNeighborsClassifier())

for i in range(7):
    committee[i].fit(X_train_init, y_train_init)

In [37]:
AL_type = input("Type 'stream' for stream based Active Learning\nType 'pool' for pool based Active Learning\nCASE INSENSITIVE\n")

query_type = input("Input the query strategy:\n'uncertainty' for Uncertainty sampling\n'QBC' for Query by Committee\n'diversity' for Diversity sampling\nCASE INSENSITIVE\n")

X_train_final = X_train_init
y_train_final = y_train_init

if query_type.lower() == 'uncertainty':
    mode = input("Input the mode:\n'1' - Least Confident\n'2' - Margin\n'3' - Entropy\n")
    mode = int(mode)
    X, Y = uncertainty(model, X_unlabelled, oracle, mode, AL_type)
    X_train_final = X_train_final.append(X)
    y_train_final = y_train_final.append(Y)

if query_type.lower() == 'qbc':
    mode = input("Input the mode:\n'1' - Entropy\n'2' - KL divergence\n")
    mode = int(mode)
    X, Y = qbc(committee, X_unlabelled, oracle, mode, AL_type)
    X_train_final = X_train_final.append(X)
    y_train_final = y_train_final.append(Y)
    
if query_type.lower() == 'diversity':
    X, Y = div(model, X_unlabelled, oracle, AL_type)
    X_train_final = X_train_final.append(X)
    y_train_final = y_train_final.append(Y)
    
y_train_final = y_train_final.astype('int')

Type 'stream' for stream based Active Learning
Type 'pool' for pool based Active Learning
CASE INSENSITIVE
pool
Input the query strategy:
'uncertainty' for Uncertainty sampling
'QBC' for Query by Committee
'diversity' for Diversity sampling
CASE INSENSITIVE
diversity


In [38]:
X_train_final.shape

(27, 13)

In [39]:
ga = LogisticRegression()
ga.fit(X_train_final, y_train_final)

prediction = ga.predict(X_test)
accuracy_score(y_test, prediction)

0.6666666666666666